[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Pruebas Unitarias en Python

La biblioteca estándar de Python incluye el módulo `unittest`, un marco de pruebas inspirado en JUnit de Java. Este módulo proporciona herramientas para construir y ejecutar pruebas, verificar condiciones (aserciones) y agrupar pruebas en suites.

## Objetivos del Cuaderno

1. Entender la estructura básica de una prueba unitaria (`TestCase`).
2. Aprender a usar aserciones (`assertEqual`, `assertTrue`, etc.).
3. Gestionar el ciclo de vida de las pruebas (`setUp`, `tearDown`).
4. Aislar dependencias usando `unittest.mock`.

## 1. Fundamentos de `unittest` y `TestCase`

Para crear pruebas, definimos una clase que hereda de `unittest.TestCase`. Cada método que comienza con `test_` se ejecuta automáticamente como una prueba.

In [ ]:
import unittest

# Código a probar (normalmente estaría en otro módulo)
def suma(a, b):
    return a + b

def division(a, b):
    if b == 0:
        raise ValueError("No se puede dividir por cero")
    return a / b

class TestOperaciones(unittest.TestCase):
    
    def test_suma_positivos(self):
        """Prueba suma de números positivos"""
        resultado = suma(10, 5)
        self.assertEqual(resultado, 15)

    def test_suma_negativos(self):
        """Prueba suma de números negativos"""
        resultado = suma(-1, -1)
        self.assertEqual(resultado, -2)

# Ejecución de las pruebas en el cuaderno
# argv=['first-arg-is-ignored'], exit=False es necesario en Jupyter
unittest.main(argv=['first-arg-is-ignored'], exit=False)

## 2. Aserciones Comunes y Manejo de Excepciones

`unittest` provee numerosos métodos para verificar resultados. Además de la igualdad, podemos verificar condiciones booleanas, pertenencia y, crucialmente, si se lanzan las excepciones esperadas.

In [ ]:
class TestAserciones(unittest.TestCase):

    def test_booleanos(self):
        # Verifica que una expresión sea True
        self.assertTrue(5 > 3)
        # Verifica que una expresión sea False
        self.assertFalse(1 > 10)

    def test_pertenencia(self):
        # Verifica si un elemento está en una colección
        lista = [1, 2, 3]
        self.assertIn(2, lista)
        self.assertNotIn(99, lista)

    def test_excepcion(self):
        # Verifica que el bloque de código lance la excepción esperada
        with self.assertRaises(ValueError):
            division(10, 0)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

## 3. Fixtures: Preparación y Limpieza

A menudo necesitamos configurar el estado antes de una prueba (crear archivos, conectar a BBDD simuladas) y limpiar después. Para esto usamos `setUp` (se corre antes de *cada* test) y `tearDown` (se corre después de *cada* test).

In [ ]:
class TestConRecursos(unittest.TestCase):

    def setUp(self):
        # Se ejecuta ANTES de cada método de prueba
        print("\n[setUp] Inicializando recursos...")
        self.datos = [1, 2, 3]

    def tearDown(self):
        # Se ejecuta DESPUÉS de cada método de prueba
        print("[tearDown] Limpiando recursos...")
        del self.datos

    def test_modificar_datos(self):
        print("  Ejecutando test_modificar_datos")
        self.datos.append(4)
        self.assertEqual(len(self.datos), 4)

    def test_datos_originales(self):
        # Comprueba que setUp reinició la lista
        print("  Ejecutando test_datos_originales")
        self.assertEqual(len(self.datos), 3)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

## 4. El módulo `unittest.mock`

Las pruebas unitarias deben ser aisladas. Si tu código depende de una API externa, una base de datos o el sistema de archivos, no quieres que tus pruebas fallen porque internet se cayó.

Para esto usamos `Mock` y `patch` para sustituir dependencias reales con objetos simulados.

In [ ]:
from unittest.mock import Mock

# Imaginemos una clase que hace llamadas lentas
class ServicioExterno:
    def obtener_usuario(self, user_id):
        # Simula una llamada lenta a API
        import time
        time.sleep(5) 
        return {"id": user_id, "nombre": "Real User"}

# Creamos un Mock para reemplazarla
mock_servicio = Mock()

# Configuramos el comportamiento del mock
mock_servicio.obtener_usuario.return_value = {"id": 1, "nombre": "Test User"}

# Usamos el mock
resultado = mock_servicio.obtener_usuario(1)

print(f"Resultado simulado: {resultado}")

# Verificamos que se llamó correctamente
mock_servicio.obtener_usuario.assert_called_with(1)

### Uso de `@patch`

`patch` es un decorador potente que intercepta dónde se busca un objeto y lo reemplaza por un mock durante la duración de la prueba.

In [ ]:
from unittest.mock import patch
import os

# Una función que usa os.getcwd
def obtener_directorio_trabajo():
    return os.getcwd()

class TestPatching(unittest.TestCase):

    # Reemplazamos 'os.getcwd' solo dentro de este test
    @patch('os.getcwd')
    def test_directorio(self, mock_getcwd):
        # Configuramos el mock que se pasa como argumento
        mock_getcwd.return_value = '/ruta/falsa'
        
        resultado = obtener_directorio_trabajo()
        
        self.assertEqual(resultado, '/ruta/falsa')
        mock_getcwd.assert_called_once()

unittest.main(argv=['first-arg-is-ignored'], exit=False)

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2017-2026.</p>